In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8n.pt')

 # Train the model
results = model.train(data='VOC.yaml', epochs=100, imgsz=640, resume=True)

In [ ]:
import os

model_name = 'yolo_voc'
for folders in os.listdir(f'data/frames'):
    results = model.predict(f'data/frames/{folders}', save_conf=True)
    predictions = {}
    for i,r in enumerate(results):
        if not os.path.exists(f'data/results/{model_name}/{folders}'):
            os.makedirs(f'data/results/{model_name}/{folders}')
        r.save(f'data/results/{model_name}/{folders}/result_{i}.jpg')
        for box in r.boxes:
            if len(box.cls):
                for i,c in enumerate(box.cls):
                    label = model.names[int(c)]
                    if label not in predictions:
                        predictions[label] = box.conf[i]
                    else:
                        predictions[label] = max(predictions[label], box.conf[i])
    # sort the predictions by confidence
    predictions = dict(sorted(predictions.items(), key=lambda item: item[1], reverse=True))
    if not os.path.exists(f'data/results/{model_name}/{folders}'):
        os.makedirs(f'data/results/{model_name}/{folders}')
    with open(f'data/results/{model_name}/{folders}/results.txt', 'w') as f:
        for k,v in predictions.items():
            f.write(f'{k}: {v}\n')